In [4]:
# libraries that will be used for the project
import numpy as np
import tensorflow as tf
#import matplotlib.pyplot as plt

import os
import re
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

## Preparing Dataset

In [5]:
def load_data(file):
    with open(file) as cap_file:
        caption = cap_file.readlines()
        cap_map = {}
        text_data = []
        skip_img = set()
        
    for i in caption:
        i = i.rstrip('\n')
        img, cap = line.split('\t')
        
        img = img.split("#")[0]
        img = os.path.join(img_path, img.strip())
        
        tokens = cap.strip().split()
        if len(token) < 5 or len(token) > 25:
            skip_img.add(img)
            continue
        if img.endwith('jpg') and img not in skip_img:
            cap =  "<start> " + caption.strip() + " <end>"
            text_data.append(cap)
        if img_name in caption_mapping:
            cap_map[img].append(cap)
        else:
            cap_map[img] = [cap]
            
    for img in skip_img:
        if img in cap_map:
            del cap_map[img]
        